<h2>Seminar 2020 - MultiThread & Pandas</h2>

In [5]:
### import section ###
from Utils.Preprocess import Preprocess
from Algorithm.ThreadManager import ThreadManager
from Utils.Log import writer, logger
from Utils.FileUtils import FileUtils
# change logger severity
logger.set_logger_severity('debug')

In [6]:
headers = ['Id', 'OverallQual', 'YearBuilt', 'OverallCond', 'OpenPorchSF']
df1 = FileUtils.read_data_frame_from_path('Data/train_1.csv', headers)
df2 = FileUtils.read_data_frame_from_path('Data/train_2.xlsx', headers)
df3 = FileUtils.read_data_frame_from_path('Data/train_targets.csv')
df_total = df1.append(df2).merge(df3, on='Id', how='left').fillna("None")
X_train, X_test, y_train, y_test = Preprocess.split_train_test_by_pandas(df_total, 'SalePrice')

params = [{"fit_intercept": True},  {'normalize': False, "fit_intercept": False}]
ThreadManager.running_threads_args(X_train, y_train, X_test, y_test, params)
ThreadManager.wait_for_all_threads()
best_result, best_params = ThreadManager.return_best_score(params)

writer.info(f'The best score is {best_result} with these params: {best_params}')

2020-03-07 18:12:54,949 Thread [{'fit_intercept': True}] is running
2020-03-07 18:12:54,953 Thread [{'normalize': False, 'fit_intercept': False}] is running
2020-03-07 18:12:54,954 set of params: {'fit_intercept': True} scored 1959089299.5312617
2020-03-07 18:12:54,957 set of params: {'normalize': False, 'fit_intercept': False} scored 1969209681.4041615
2020-03-07 18:12:54,958 Thread [{'fit_intercept': True}] is finished
2020-03-07 18:12:54,958 Thread [{'normalize': False, 'fit_intercept': False}] is finished
2020-03-07 18:12:54,962 The best score is 1959089299.5312617 with these params: {'fit_intercept': True}
